<a href="https://colab.research.google.com/github/sahupra1357/LLMAgents/blob/main/gorq_benchmarking_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/groq/groq-api-cookbook/blob/main/tutorials/benchmarking-rag-langchain/benchmarking_rag.ipynb

In [ ]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
print(GROQ_API_KEY)

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


In [ ]:
!pip install langchain -q
!pip install langchain_chroma -q
!pip install langchain_community -q
!pip install langchain_groq -q
!pip install grandalf -q
!pip install numpy -q
!pip install pandas -q
!pip install sentence-transformers -q

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To suppress huggingface warnings
os.environ["OPENAI_API_KEY"] = "userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install transformers numpy

In [ ]:
!pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.2 MB/s eta 0:00:00


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

#embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")
rag_llm = ChatGroq(model="llama3-8b-8192") # Used for RAG
qa_llm = ChatGroq(model="llama3-70b-8192", temperature=0.1) # Used to create eval dataset
benchmark_llm = ChatGroq(model="llama3-70b-8192") # Used to evaluate (Judge)

In [ ]:
# Download Paul Graham's Essay
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2025-03-26 03:50:43--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.02s   

2025-03-26 03:50:44 (3.27 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [ ]:
!ls data/paul_graham/paul_graham_essay.txt

data/paul_graham/paul_graham_essay.txt


In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Use popular github repo instead
loader = DirectoryLoader("./data/paul_graham/", use_multithreading=True, loader_cls=TextLoader)
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        "",
        # For multilingual/non-english text
        # "\u200b",  # Zero-width space
        # "\uff0c",  # Fullwidth comma
        # "\u3001",  # Ideographic comma
        # "\uff0e",  # Fullwidth full stop
        # "\u3002",  # Ideographic full stop
    ],
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
documents = loader.load_and_split(text_splitter=text_splitter) # Load text
vectorstore = Chroma.from_documents(documents, embedding=embed_model, collection_name="groq_rag")
retriever = vectorstore.as_retriever()
print(f"Documents indexed: {len(documents)}")

Documents indexed: 27


In [ ]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7a6cf3cbe250>, search_kwargs={})

In [ ]:
await retriever.ainvoke("What did paul graham do growing up?")

[Document(id='ffb5abe6-650a-480a-b0e1-156cf6794ea5', metadata={'source': 'data/paul_graham/paul_graham_essay.txt'}, page_content="The subset I would build as an open source project was the new Lisp, whose parentheses I now wouldn't even have to hide. A lot of Lisp hackers dream of building a new Lisp, partly because one of the distinctive features of the language is that it has dialects, and partly, I think, because we have in our minds a Platonic form of Lisp that all existing dialects fall short of. I certainly did. So at the end of the summer Dan and I switched to working on this new dialect of Lisp, which I called Arc, in a house I bought in Cambridge.\n\nThe following spring, lightning struck. I was invited to give a talk at a Lisp conference, so I gave one about how we'd used Lisp at Viaweb. Afterward I put a postscript file of this talk online, on paulgraham.com, which I'd created years before using Viaweb but had never used for anything. In one day it got 30,000 page views. Wha

In [ ]:
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict

RAG_SYSTEM_PROMPT = """\
You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context given within delimiters to answer the human's questions.
```
{context}
```
If you don't know the answer, just say that you don't know.\
""" # adapted from https://smith.langchain.com/hub/rlm/rag-prompt-llama3

RAG_HUMAN_PROMPT = "{input}"

RAG_PROMPT = ChatPromptTemplate.from_messages([
    ("system", RAG_SYSTEM_PROMPT),
    ("human", RAG_HUMAN_PROMPT)
])

def format_docs(docs: List[Document]):
    """Format the retrieved documents"""
    return "\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs, # Use retriever to retrieve docs from vectorstore -> format the documents into a string
        "input": RunnablePassthrough() # Propogate the 'input' variable to the next step
    }
    | RAG_PROMPT # format prompt with 'context' and 'input' variables
    | rag_llm # get response from LLM using the formatteed prompt
    | StrOutputParser() # Parse through LLM response to get only the string response

)

In [ ]:
await rag_chain.ainvoke("What did paul graham do growing up?")

"According to the provided context, Paul Graham grew up in a family where his parents were both artists, and he was homeschooled by his mother. He was an art student and didn't learn how to program computers until he was 20 years old. He then went to RISD (Rhode Island School of Design) but dropped out after a year due to financial issues. He then got a job at Interleaf, a software company, where he learned some programming skills."

In [ ]:
len(documents)

27

In [ ]:
documents[0]

Document(metadata={'source': 'data/paul_graham/paul_graham_essay.txt'}, page_content='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used

In [ ]:
# Create questions for eval pipeline
from typing import TypedDict
# Response object structure
class QAResponse(TypedDict):
    question_1: str
    question_2: str
    question_3: str

QA_HUMAN_PROMPT = """\
You are a Teacher/ Professor. Your task is to setup questions for an upcoming \
quiz/examination. The questions should be diverse in nature across the document. \
Given the context information and not prior knowledge, generate only questions based on the below context. \
Restrict the questions to the context information provided within the delimiters.
```
{text}
```
Output the questions in JSON format with the keys question_1, question_2 and question_3 \
and make sure to escape any special characters to output clean, valid JSON.\
""" # adapted from https://arize.com/blog/evaluate-rag-with-llm-evals-and-benchmarking/

QA_PROMPT = ChatPromptTemplate.from_messages([
    ("human", QA_HUMAN_PROMPT)
])

qa_chain = (
{"text": RunnablePassthrough()}
| QA_PROMPT
| qa_llm.with_structured_output(method='json_mode', schema=QAResponse) # either 'json_mode' or 'function_calling' to get responses always in JSON format
)

In [ ]:
texts = [doc.page_content for doc in documents] # Create a list with all the text from the document chunks in the vectorstore

questions: List[Dict] = await qa_chain.abatch(texts)

In [ ]:
questions

[{'question_1': 'What were the two main things the author worked on outside of school before college?',
  'question_2': 'What was the language used to write programs on the IBM 1401?',
  'question_3': "What was the name of the microcomputer the author's friend built from a kit sold by Heathkit?"},
 {'question_1': 'What was the author planning to study in college initially?',
  'question_2': 'What was the name of the novel that inspired the author to work on AI?',
  'question_3': 'What was the name of the programming language the author had to learn to work on AI?'},
 {'question_1': 'What were the three graduate schools the author applied to?',
  'question_2': 'What was the author’s realization about AI during the first year of grad school?',
  'question_3': 'What was the author’s focus on after realizing the limitations of AI?'},
 {'question_1': 'What was the realization the author had while visiting the Carnegie Institute in 1988?',
  'question_2': "What was the author's advisor's nam

In [ ]:
print(f"From document: \n{texts[0]}\n")
print(f"Questions generated:")
for i, q in enumerate(questions[0].values(), 1): print(f'{i}: {q}')

From document: 
What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then s

In [ ]:
from pydantic import BaseModel

# Response object structure
class EvalResponse(BaseModel):
    score: int
    explanation: str

EVAL_HUMAN_PROMPT = """\
You are given a question, an answer and reference text within marked delimiters. \
You must determine whether the given answer correctly answers the question based on the reference text. Here is the data:
```Question
{question}
```
```Reference
{context}
```
```Answer
{answer}
```
Respond with a valid JSON object containing two fields:
{{
    "score": "int: a score between 0-10, 10 being highest, on whether the question is correctly and fully answered by the answer",
    "explanation": "str: Provide an explanation as to why the score was given."
}}
Make sure to escape any special characters to output clean, valid JSON.\
"""
# adapted from https://docs.arize.com/phoenix/evaluation/how-to-evals/running-pre-tested-evals/q-and-a-on-retrieved-data

EVAL_PROMPT = ChatPromptTemplate.from_messages([
    ("human", EVAL_HUMAN_PROMPT)
])

eval_chain = (
    {
    "context": RunnablePassthrough(), # Propogate all input vars to next step in pipeline
    "question": RunnablePassthrough(),
    "answer": RunnablePassthrough(),
    }
    | EVAL_PROMPT
    | benchmark_llm.with_structured_output(schema=EvalResponse, method='json_mode') # Parse response according to EvalResponse object
)

In [ ]:
q1 = questions[-1]['question_1']
t1 = texts[-1]


print(f"Question: {q1}")
a1 = await rag_chain.ainvoke(q1)
print(f"Answer: {a1}")
eval_input = {
    'context': t1,
    'question': q1,
    'answer': a1
}
response = await eval_chain.ainvoke(eval_input)

Question: What is the problem that occurs when you both write essays and run a forum, according to the passage?
Answer: According to the passage, the problem that occurs when you both write essays and run a forum is that people post highly imaginative misinterpretations of your essays on the forum, and you have to respond to them because the assumption is that you're present in the conversation, but responding to them encourages more misinterpretations.


In [ ]:
eval_input

{'context': "[17] Another problem with HN was a bizarre edge case that occurs when you both write essays and run a forum. When you run a forum, you're assumed to see if not every conversation, at least every conversation involving you. And when you write essays, people post highly imaginative misinterpretations of them on forums. Individually these two phenomena are tedious but bearable, but the combination is disastrous. You actually have to respond to the misinterpretations, because the assumption that you're present in the conversation means that not responding to any sufficiently upvoted misinterpretation reads as a tacit admission that it's correct. But that in turn encourages more; anyone who wants to pick a fight with you senses that now is their chance.\n\n[18] The worst thing about leaving YC was not working with Jessica anymore. We'd been working on YC almost the whole time we'd known each other, and we'd neither tried nor wanted to separate it from our personal lives, so lea

In [ ]:
print(dir(response.json))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer__', '__pydantic_validator__', '__reduce__', '__reduce_ex__', '__replace__', '

In [ ]:
print(response.explanation)

The answer accurately summarizes the problem described in the passage, which is that when you write essays and run a forum, people post misinterpretations of your essays and you have to respond to them, and this leads to more misinterpretations.


In [ ]:
print(response.score)

10


In [ ]:
print("---------------------")
print(f"Score: {response.score}")
print(f"Explanation: {response.explanation}")
print("---------------------")

---------------------
Score: 10
Explanation: The answer accurately summarizes the problem described in the passage, which is that when you write essays and run a forum, people post misinterpretations of your essays and you have to respond to them, and this leads to more misinterpretations.
---------------------


In [ ]:
# Lets do it for all now
from typing import TypedDict
from time import time

class EvalResult(TypedDict): # For type hinting
    question: str
    answer: str
    context: str
    score: int # Score between 0 - 10
    explanation: str # Explanation on why the score was given

async def evaluate(questions: List[Dict] = questions, texts: List[str] = texts) -> List[EvalResult]:
    # Prepare inputs
    batch_rag_inputs: List[Dict] = []
    evals: List[Dict] = []
    for q_dict, context in zip(questions, texts):
        for question in q_dict.values():
            batch_rag_inputs.append(question)
            evals.append({'question': question, 'context': context})

    print(f"Running RAG pipeline for {len(batch_rag_inputs)} questions")
    start = time()
    answers = await rag_chain.abatch(batch_rag_inputs, config={'max_concurrency': 2}) # Reduce concurrency to avoid hitting rate limits
    end = time()
    print(f"Time taken: {end - start}")

    # Update eval_input with the answers from the rag_chain
    for eval_input, answer in zip(evals, answers):
        eval_input.update({'answer': answer})

    # Run eval_chain to get evaluation
    print(f"Evaluating RAG pipeline...")
    start = time()
    batch_score_explanations = await eval_chain.abatch(evals, config={'max_concurrency': 2}) # Pass in eval which contains List of 'answer', 'context', 'question'
    end = time()
    print(f"Time taken: {end - start}")

    # Update eval variable with the score and explanation
    for eval, score_exp_dict in zip(evals, batch_score_explanations):
        eval.update({
            #'score': score_exp_dict['score'],
            #'explanation': score_exp_dict['explanation']
            'score': score_exp_dict.score,
            'explanation': score_exp_dict.explanation
        })

    return evals

In [ ]:
evaluations = await evaluate(questions[:2], texts[:2]) # Remove the `:2` to evaluate all the questions on all your data

Running RAG pipeline for 6 questions
Time taken: 105.0009412765503
Evaluating RAG pipeline...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jm13tccdez3atcxbyd8cf3y1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5674, Requested 2683. Please try again in 23.566999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
csv_file = 'evaluations.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Context', 'Score', 'Question', 'Answer', 'Explanation'])
    for eval in evaluations:
        writer.writerow([eval['context'], eval['score'], eval['question'], eval['answer'], eval['explanation']])

print(f"Evaluations saved to {csv_file}")